<a href="https://colab.research.google.com/github/gmauricio-toledo/Curso-Python-2023/blob/main/Notebooks/09-M%C3%A9todos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# Eliminación Gaussiana

En todas las implementaciones, observar el argumento por default en el llamado a la función

Implementación usando vectorización de Numpy

In [ ]:
def eliminacion_gaussiana(A,b,resultados_parciales=False):
    '''
    Esta función calcula la solución de un sistema de ecuaciones Ax=b usando Eliminación Gaussiana para llevar la
    matriz A a su forma triangular superior y después realiza la sustitución hacia atrás.
    '''
    n = A.shape[0]
    assert b.shape[0] == n  # Aquí nos aseguramos que la forma de b sea compatible con la forma de A
    A = np.hstack((A,b.reshape(-1,1))) # Formamos la matriz aumentada del sistema
    if resultados_parciales: print("Inicial:\n", A) # Podemos escribir el if en una sola línea
    for k in range(n-1): # Recorremos cada columna
        for i in range(k+1,n): # Recorremos de la diagonal hacia adelante
            ratio = A[i,k]/A[k,k]
            A[i] = A[i]-ratio*A[k] # En este punto usamos la vectorización
            if resultados_parciales: print(A)
    if resultados_parciales: print("Forward elimination:\n",A)
    variables = A[:,n]
    for k in reversed(range(n)): # Realizamos la sustitución hacia atras
        for j in range(k+1,n):
            variables[k] = variables[k] - A[k,j]*variables[j] # Otra vez, usamos vectorización
        variables[k] = variables[k]/A[k,k]
    return variables

Ahora, incluyamos el pivoteo parcial. Esto lo hacemos manipulando los índices de los arreglos de numpy.

In [ ]:
def eliminacion_gaussiana_pp(A,b,resultados_parciales=False):
    '''
    Esta función calcula la solución de un sistema de ecuaciones Ax=b usando Eliminación Gaussiana con
    pivoteo parcial para llevar la matriz A a su forma triangular superior y después realiza la
    sustitución hacia atrás.
    '''
    n = A.shape[0]
    assert b.shape[0] == n
    A = np.hstack((A,b.reshape(-1,1))) # Formamos la matriz aumentada del sistema
    if resultados_parciales: print("Inicial:\n", A)
    for k in range(n-1):
        idx_max = np.argmax(np.abs(A[k:,k])) + k
        new_idxs = list(range(A.shape[0]))
        new_idxs[k] = idx_max
        new_idxs[idx_max] = k
        A = A[new_idxs]
        if resultados_parciales: print("cambio de renglones:\n",A)
        for i in range(k+1,n):
            ratio = A[i,k]/A[k,k]
            A[i] = A[i]-ratio*A[k] # En este punto usamos la vectorización
            if resultados_parciales: print(A)
    if resultados_parciales: print("Forward elimination:\n",A)
    variables = A[:,n]
    for k in reversed(range(n)):
        for j in range(k+1,n):
            variables[k] = variables[k] - A[k,j]*variables[j] # Otra vez, usamos vectorización
        variables[k] = variables[k]/A[k,k]
    return variables

## Aspectos importantes

¿Cómo juntamos arreglos en uno solo?

In [ ]:
A = np.array([[3,2,1],[5,3,4],[1,1,-1]],dtype=float)
b = np.array([1,2,1],dtype=float)

print(A)
print(b.reshape(-1,1))
np.hstack((A,b.reshape(-1,1)))

¿Cómo intercambiamos dos líneas de un arreglo?

In [ ]:
A = np.array([[3,2,1],[5,3,4],[1,1,-1]],dtype=float)
print(A)
new_idxs = [2,1,0]
A = A[new_idxs]  # Aquí hacemos la permutación
print(A)

## Ejemplo 1

### Usando nuestra implementación

In [ ]:
A = np.array([[3,2,1],[5,3,4],[1,1,-1]],dtype=float)
b = np.array([1,2,1],dtype=float)
print(A,"\n\n",b)

In [ ]:
eliminacion_gaussiana(A,b)

In [ ]:
eliminacion_gaussiana_pp(A,b,resultados_parciales=True)

### Usando implementaciones de Python

Usando numpy.

In [ ]:
import numpy as np

np.linalg.solve(A,b)

In [ ]:
np.linalg.inv(A)@b

Usando [`scipy`](https://docs.scipy.org/doc/scipy/tutorial/index.html#user-guide)

In [ ]:
import scipy

scipy.linalg.solve(A,b)

En el siguiente ejemplo vemos que, en general, necesitamos el método con pivoteo parcial

In [ ]:
A = np.array([[0,2,1],[5,0,4],[1,1,-1]],dtype=float)
b = np.array([1,2,1],dtype=float)
print(A,"\n\n",b,"\n")

print(f"Eliminación Gaussiana sin pivoteo parcial\n{eliminacion_gaussiana(A,b)}\n")
print(f"Eliminación Gaussiana con pivoteo parcial\n{eliminacion_gaussiana_pp(A,b)}\n")
print(f"Usando el solver de numpy\n{np.linalg.solve(A,b)}")

Comparemos el tiempo de ejecución de ambas opciones (la propia y la de Python):

In [ ]:
import time
import scipy

# --- opción 1 ---
inicio = time.time()
sol = eliminacion_gaussiana(A,b)
final = time.time()
print(f"Duración (EG): {final-inicio} segundos")

# --- opción 2 ---
inicio = time.time()
sol = eliminacion_gaussiana_pp(A,b)
final = time.time()
print(f"Duración (EGpp): {final-inicio} segundos")

# --- opción 3 ---
inicio = time.time()
sol = np.linalg.solve(A,b)
final = time.time()
print(f"Duración (numpy): {final-inicio} segundos")

# --- opción 4 ---
inicio = time.time()
sol = scipy.linalg.solve(A,b)
final = time.time()
print(f"Duración (scipy): {final-inicio} segundos")

Ahora, veamos con un ejemplo grande

In [ ]:
import numpy as np

np.random.seed(22)
Ab = np.random.normal(size=(200,201))

A = Ab[:,:-1].copy()
b = Ab[:,-1].copy()
print(f"Determinante de A: {np.linalg.det(A)}")

In [ ]:
import time
import scipy

# --- opción 2 ---
inicio = time.time()
sol = eliminacion_gaussiana_pp(A,b)
final = time.time()
print(f"Duración (EGpp): {final-inicio} segundos")

# --- opción 3 ---
inicio = time.time()
sol = np.linalg.solve(A,b)
final = time.time()
print(f"Duración (numpy): {final-inicio} segundos")

# --- opción 4 ---
inicio = time.time()
sol = scipy.linalg.solve(A,b)
final = time.time()
print(f"Duración (scipy): {final-inicio} segundos")